In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
#!pip install pydantic==1.10.8

In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

In [4]:
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

In [5]:
chain = prompt | model | output_parser

In [8]:
chain.invoke({"topic": "death"})

'Why did the skeleton go to the party alone? \n\nBecause he had no body to go with him!'

In [9]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

In [12]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["she went to school everyday", "The new movie is coming soon"],
    embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

In [16]:
retriever.get_relevant_documents("she went to school everyday")

[Document(page_content='she went to school everyday'),
 Document(page_content='The new movie is coming soon')]

In [15]:
retriever.get_relevant_documents("The new movie is coming soon")

[Document(page_content='The new movie is coming soon'),
 Document(page_content='she went to school everyday')]

In [17]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [18]:
from langchain.schema.runnable import RunnableMap

In [19]:
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [20]:
chain.invoke({"question": "where did she go everyday?"})

'She went to school everyday.'

In [21]:
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

In [22]:
inputs.invoke({"question": "where did she go everyday?"})

{'context': [Document(page_content='she went to school everyday'),
  Document(page_content='The new movie is coming soon')],
 'question': 'where did she go everyday?'}